In [7]:
import copy
import torch

from datasets import load_dataset, DatasetDict, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)
from data import RationalWeight
from data import Enumerator

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}
data_files = {'train': 'train.parquet', 'test': 'test.parquet'}
dataset = load_dataset('../datasets/imdbstfd', data_files=data_files)
dataset = dataset.shuffle(seed=88)
model = AutoModelForSequenceClassification.from_pretrained(
    "../models/bert-base-uncased-IMDB-retrain", num_labels=2, id2label=id2label, label2id=label2id
)
model.eval()
tokenizer = AutoTokenizer.from_pretrained("../models/bert-base-uncased-IMDB-retrain", use_fast=True)

In [9]:
for name, param in model.named_parameters():
    print(name)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

In [10]:
nameExclude = ["99"]
exclude = []
for name, param in model.named_parameters():
    for excludeName in nameExclude:
        if excludeName in name:
            exclude.append(name)

layers = list(filter(lambda x: x not in exclude, [name for name, param in model.named_parameters()]))

len(layers)

201

In [11]:
N=6

In [12]:
# PATH = "../model-prun/models_3_3_095"
PATH = "../model-prun/reprun-models"

# models = torch.load(PATH, map_location=torch.device('cpu'), weights_only=False)
models = torch.load(PATH, weights_only=False)
models

[{'prunModel': <data.RationalWeightClass.RationalWeight at 0x13532d6a590>,
  'data': {'sigma': 1,
   'scale': 1.5,
   'inside': True,
   'RESULT': {'Accuracy': 0.9,
    'ROC_AUC_score': 0.8976982097186701,
    'F1': 0.8823529411764706,
    'Precision': 0.8823529411764706,
    'Recall': 0.8823529411764706},
   'Pruning_layers': ['bert.encoder.layer.0.attention.self.key.bias',
    'bert.encoder.layer.0.intermediate.dense.bias',
    'bert.encoder.layer.1.attention.self.query.weight',
    'bert.encoder.layer.1.attention.self.key.bias',
    'bert.encoder.layer.2.attention.self.key.bias',
    'bert.encoder.layer.2.attention.output.LayerNorm.bias',
    'bert.encoder.layer.2.intermediate.dense.bias',
    'bert.encoder.layer.2.output.LayerNorm.bias',
    'bert.encoder.layer.3.attention.self.key.bias',
    'bert.encoder.layer.3.intermediate.dense.bias',
    'bert.encoder.layer.4.attention.self.key.bias',
    'bert.encoder.layer.4.attention.output.dense.bias',
    'bert.encoder.layer.4.output.den

In [13]:
prunLayers = [models[i]['data']['Pruning_layers'] for i in range(N)] #199

In [14]:
[models[i]['data']['RESULT'] for i in range(N)] #199

[{'Accuracy': 0.9,
  'ROC_AUC_score': 0.8976982097186701,
  'F1': 0.8823529411764706,
  'Precision': 0.8823529411764706,
  'Recall': 0.8823529411764706},
 {'Accuracy': 0.9,
  'ROC_AUC_score': 0.9437340153452685,
  'F1': 0.8888888888888888,
  'Precision': 0.8421052631578947,
  'Recall': 0.9411764705882353},
 {'Accuracy': 0.9,
  'ROC_AUC_score': 0.9002557544757034,
  'F1': 0.8823529411764706,
  'Precision': 0.8823529411764706,
  'Recall': 0.8823529411764706},
 {'Accuracy': 0.9,
  'ROC_AUC_score': 0.9053708439897699,
  'F1': 0.8823529411764706,
  'Precision': 0.8823529411764706,
  'Recall': 0.8823529411764706},
 {'Accuracy': 0.9,
  'ROC_AUC_score': 0.9258312020460358,
  'F1': 0.8888888888888888,
  'Precision': 0.8421052631578947,
  'Recall': 0.9411764705882353},
 {'Accuracy': 0.9,
  'ROC_AUC_score': 0.9053708439897699,
  'F1': 0.8823529411764706,
  'Precision': 0.8823529411764706,
  'Recall': 0.8823529411764706}]

In [15]:
[len(models[i]['data']['Pruning_layers']) for i in range(N)] #199

[42, 39, 27, 31, 38, 24]

In [16]:
for i in range(N):
    layers1_1 = models[i]['data']['Pruning_layers']
    print(models[i]['prunModel'].zeroCounter(layers)[-1])

{'perZerosInModel': tensor(0.1887, device='cuda:0'), 'allParam': 109483778, 'allZero': tensor(20654641, device='cuda:0')}
{'perZerosInModel': tensor(0.1849, device='cuda:0'), 'allParam': 109483778, 'allZero': tensor(20240161, device='cuda:0')}
{'perZerosInModel': tensor(0.1849, device='cuda:0'), 'allParam': 109483778, 'allZero': tensor(20242962, device='cuda:0')}
{'perZerosInModel': tensor(0.2055, device='cuda:0'), 'allParam': 109483778, 'allZero': tensor(22496536, device='cuda:0')}
{'perZerosInModel': tensor(0.1919, device='cuda:0'), 'allParam': 109483778, 'allZero': tensor(21008161, device='cuda:0')}
{'perZerosInModel': tensor(0.1849, device='cuda:0'), 'allParam': 109483778, 'allZero': tensor(20243155, device='cuda:0')}


In [17]:
for i in range(N):
    layers1_1 = models[i]['data']['Pruning_layers']
    print(models[i]['prunModel'].zeroCounter(layers1_1)[-2])

{'layer': 'bert.encoder.layer.11.output.dense.bias', 'countZero': tensor(553, device='cuda:0'), 'percentZero': tensor(0.7201, device='cuda:0'), 'numParamLayer': 768}
{'layer': 'bert.encoder.layer.11.output.dense.bias', 'countZero': tensor(553, device='cuda:0'), 'percentZero': tensor(0.7201, device='cuda:0'), 'numParamLayer': 768}
{'layer': 'bert.pooler.dense.bias', 'countZero': tensor(704, device='cuda:0'), 'percentZero': tensor(0.9167, device='cuda:0'), 'numParamLayer': 768}
{'layer': 'bert.pooler.dense.bias', 'countZero': tensor(704, device='cuda:0'), 'percentZero': tensor(0.9167, device='cuda:0'), 'numParamLayer': 768}
{'layer': 'bert.pooler.dense.bias', 'countZero': tensor(766, device='cuda:0'), 'percentZero': tensor(0.9974, device='cuda:0'), 'numParamLayer': 768}
{'layer': 'bert.encoder.layer.11.attention.self.value.bias', 'countZero': tensor(754, device='cuda:0'), 'percentZero': tensor(0.9818, device='cuda:0'), 'numParamLayer': 768}


In [18]:
for i in range(N):
    print(models[i]['data'])
    print(models[i]['prunModel'].zeroCounter(layers)[-1])

{'sigma': 1, 'scale': 1.5, 'inside': True, 'RESULT': {'Accuracy': 0.9, 'ROC_AUC_score': 0.8976982097186701, 'F1': 0.8823529411764706, 'Precision': 0.8823529411764706, 'Recall': 0.8823529411764706}, 'Pruning_layers': ['bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.1.attention.self.query.weight', 'bert.encoder.layer.1.attention.self.key.bias', 'bert.encoder.layer.2.attention.self.key.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.intermediate.dense.bias', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.self.key.bias', 'bert.encoder.layer.3.intermediate.dense.bias', 'bert.encoder.layer.4.attention.self.key.bias', 'bert.encoder.layer.4.attention.output.dense.bias', 'bert.encoder.layer.4.output.dense.bias', 'bert.encoder.layer.5.attention.self.query.weight', 'bert.encoder.layer.5.attention.self.key.bias', 'bert.encoder.layer.5.attention.self.value.bias', '

In [19]:
[models[i]['prunModel'].zeroCounter(layers)[-1]['perZerosInModel'] for i in range(N)]

[tensor(0.1887, device='cuda:0'),
 tensor(0.1849, device='cuda:0'),
 tensor(0.1849, device='cuda:0'),
 tensor(0.2055, device='cuda:0'),
 tensor(0.1919, device='cuda:0'),
 tensor(0.1849, device='cuda:0')]

In [20]:
for i in range(N):
    layerPrun = models[i]['data']['Pruning_layers']
    avr = [models[i]['prunModel'].zeroCounter(layerPrun)[j]['percentZero'] for j in range(len(layerPrun))]
    print(models[i]['data']['sigma'], models[i]['data']['scale'], torch.mean(torch.Tensor(avr)))

1 1.5 tensor(0.7796)
1 2 tensor(0.7689)
2 1.5 tensor(0.9301)
2 2 tensor(0.9342)
3 1.5 tensor(0.9838)
3 2 tensor(0.9827)


In [21]:
print(
    'sigma=1:',
    ((len(set(prunLayers[0]) ^ set(prunLayers[1]) ^ set(prunLayers[2]))), len(set(prunLayers[0]) & set(prunLayers[1]) & set(prunLayers[2]))),
    'sigma=2:',
    ((len(set(prunLayers[3]) ^ set(prunLayers[4]) ^ set(prunLayers[5]))), len(set(prunLayers[3]) & set(prunLayers[4]) & set(prunLayers[5]))),
    'sigma=3:',
    ((len(set(prunLayers[6]) ^ set(prunLayers[7]) ^ set(prunLayers[8]))), len(set(prunLayers[6]) & set(prunLayers[7]) & set(prunLayers[8])))
)

IndexError: list index out of range

In [56]:
print(
    'scale=1:',
    ((len(set(prunLayers[0]) ^ set(prunLayers[3]) ^ set(prunLayers[6]))), len(set(prunLayers[0]) & set(prunLayers[3]) & set(prunLayers[6]))),
    'scale=1.5:',
    ((len(set(prunLayers[1]) ^ set(prunLayers[4]) ^ set(prunLayers[7]))), len(set(prunLayers[1]) & set(prunLayers[4]) & set(prunLayers[7]))),
    'scale=2:',
    ((len(set(prunLayers[2]) ^ set(prunLayers[5]) ^ set(prunLayers[8]))), len(set(prunLayers[2]) & set(prunLayers[5]) & set(prunLayers[8])))
)

scale=1: (84, 36) scale=1.5: (94, 51) scale=2: (74, 28)


In [58]:
len(set(prunLayers[0]) & set(prunLayers[3]) & set(prunLayers[6])& set(prunLayers[0]) & set(prunLayers[1]) & set(prunLayers[2]))

23

In [61]:
len(set(prunLayers[3]) & set(prunLayers[4]) & set(prunLayers[5]) & set(prunLayers[1]) & set(prunLayers[4]) & set(prunLayers[7]))

28

In [60]:
len(set(prunLayers[6]) & set(prunLayers[7]) & set(prunLayers[8])& set(prunLayers[2]) & set(prunLayers[5]) & set(prunLayers[8]))

20

In [67]:
PATH = "../model-prun/models_1s_1.5-2"
modelsDense = torch.load(PATH, weights_only=False)

In [68]:
[len(modelsDense[i]['data']['Pruning_layers']) for i in range(2)] #199 set([1,2,3]) & set([2,3,4])

[100, 74]

In [69]:
for i in range(2):
    print(modelsDense[i]['prunModel'].zeroCounter(layers)[-1])

{'perZerosInModel': tensor(0.1848, device='cuda:0'), 'allParam': 109483778, 'allZero': tensor(20237128, device='cuda:0')}
{'perZerosInModel': tensor(0.0985, device='cuda:0'), 'allParam': 109483778, 'allZero': tensor(10778722, device='cuda:0')}


In [70]:
prunLayers = [modelsDense[i]['data']['Pruning_layers'] for i in range(2)] #199

In [72]:
for i in range(2):
    layerPrun = modelsDense[i]['data']['Pruning_layers']
    avr = [modelsDense[i]['prunModel'].zeroCounter(layerPrun)[j]['percentZero'] for j in range(len(layerPrun))]
    print(modelsDense[i]['data']['sigma'], modelsDense[i]['data']['scale'], torch.mean(torch.Tensor(avr)))

1 1.5 tensor(0.7539)
1 2 tensor(0.7555)


In [73]:
[modelsDense[i]['prunModel'].zeroCounter(layers)[-1]['perZerosInModel'] for i in range(2)]

[tensor(0.1848, device='cuda:0'), tensor(0.0985, device='cuda:0')]

In [74]:
(len(set(prunLayers[0]) ^ set(prunLayers[1])), len(set(prunLayers[0]) & set(prunLayers[1])))

(62, 56)

In [75]:
len(set(prunLayers[0]) - set(prunLayers[1]))

44